In [1]:
geoserver_url="http://localhost:8080/geoserver"
geoserver_rest=geoserver_url+"/rest"
#definir los workspaces a utilizar
workspaces_use0=[("servicios_sig1","ran"),("tamaulipas_prueba","prueba")]
workspaces_use= [w[0] for w in  workspaces_use0]
workspaces_use

['servicios_sig1', 'tamaulipas_prueba']

In [2]:
import requests
todos_works=requests.get(geoserver_rest+"/workspaces.json",auth=("admin","geoserver")).json()
todos_works

{'workspaces': {'workspace': [{'name': 'cite',
    'href': 'http://localhost:8080/geoserver/rest/workspaces/cite.json'},
   {'name': 'tiger',
    'href': 'http://localhost:8080/geoserver/rest/workspaces/tiger.json'},
   {'name': 'nurc',
    'href': 'http://localhost:8080/geoserver/rest/workspaces/nurc.json'},
   {'name': 'sde',
    'href': 'http://localhost:8080/geoserver/rest/workspaces/sde.json'},
   {'name': 'it.geosolutions',
    'href': 'http://localhost:8080/geoserver/rest/workspaces/it.geosolutions.json'},
   {'name': 'topp',
    'href': 'http://localhost:8080/geoserver/rest/workspaces/topp.json'},
   {'name': 'sf',
    'href': 'http://localhost:8080/geoserver/rest/workspaces/sf.json'},
   {'name': 'tamaulipas_prueba',
    'href': 'http://localhost:8080/geoserver/rest/workspaces/tamaulipas_prueba.json'},
   {'name': 'servicios_sig1',
    'href': 'http://localhost:8080/geoserver/rest/workspaces/servicios_sig1.json'}]}}

In [3]:
disponibles=[ ww["name"] for ww in todos_works["workspaces"]["workspace"] ]
matches=set(disponibles) & set(workspaces_use)
works_to_usar=[]
for ww in todos_works["workspaces"]["workspace"]:
    for unm in matches:
        if ww["name"]==unm:
            reqds=requests.get(ww["href"],auth=("admin","geoserver"))
            works_to_usar.append(reqds.json())
            
works_to_usar

[{'workspace': {'name': 'tamaulipas_prueba',
   'isolated': False,
   'dataStores': 'http://localhost:8080/geoserver/rest/workspaces/tamaulipas_prueba/datastores.json',
   'coverageStores': 'http://localhost:8080/geoserver/rest/workspaces/tamaulipas_prueba/coveragestores.json',
   'wmsStores': 'http://localhost:8080/geoserver/rest/workspaces/tamaulipas_prueba/wmsstores.json',
   'wmtsStores': 'http://localhost:8080/geoserver/rest/workspaces/tamaulipas_prueba/wmtsstores.json'}},
 {'workspace': {'name': 'servicios_sig1',
   'isolated': False,
   'dataStores': 'http://localhost:8080/geoserver/rest/workspaces/servicios_sig1/datastores.json',
   'coverageStores': 'http://localhost:8080/geoserver/rest/workspaces/servicios_sig1/coveragestores.json',
   'wmsStores': 'http://localhost:8080/geoserver/rest/workspaces/servicios_sig1/wmsstores.json',
   'wmtsStores': 'http://localhost:8080/geoserver/rest/workspaces/servicios_sig1/wmtsstores.json'}}]

In [4]:
for wok in works_to_usar:
    datareqds=requests.get(wok["workspace"]["dataStores"],auth=("admin","geoserver") )
    datstores=datareqds.json()
    wok["list_dataStores"]=[]
    for ds in datstores["dataStores"]["dataStore"]:
        reqds=requests.get(ds["href"],auth=("admin","geoserver"))
        datastore1=reqds.json() #obj to list
        #print(datastore1["dataStore"]["featureTypes"])
        hrefs_features_req=requests.get(datastore1["dataStore"]["featureTypes"],auth=("admin","geoserver"))
        hrefs_features=hrefs_features_req.json()
        datastore1["list_featureTypes"]=[]
        for featureqq in hrefs_features["featureTypes"]["featureType"]:
            req_feature1=requests.get(featureqq["href"],auth=("admin","geoserver"))
            featureType1=req_feature1.json()
            datastore1["list_featureTypes"].append(featureType1)
        wok["list_dataStores"].append(datastore1)


In [18]:
settings_wms_viz={
    "settings": {
        "zoom": 10,
        "center": [
            -99.133209,
            19.432608
        ]
    },
    "skeleton": {
        "base": [
            "base_osm"
        ],
        "main":[]
    },
    "layers":[
        {
            "type": "tile",
            "name": "base_osm",
            "title": "Open Street Map",
            "visible": True,
            "settings": {
                "ol_source_accesor": "OSM"
            },
            "plugin_view": {}
        }
    ]
}
for w1 in works_to_usar:
    print(w1["workspace"]["name"]+"<<<--")
    layers_grupo=[]
    for d1 in w1["list_dataStores"]:
        for f1 in d1["list_featureTypes"]:
            elf=f1["featureType"]
            lay={
                "type":"wms",
                "name":elf["name"],
                "title":elf["title"],
                "visible":False,
                "opacity":1,
                "settings":{
                    "request_body":{
                        "LAYERS":elf["name"],
                        "SRS":elf["srs"],
                        "TILED":False,
                    },
                    "url":""
                }
            }
            print(elf["store"]["name"])
    settings_wms_viz["skeleton"]["main"].append(dict(title=w1["workspace"]["name"],layers=layers_grupo))

tamaulipas_prueba<<<--
tamaulipas_prueba:tamaulipas911_aut
servicios_sig1<<<--
servicios_sig1:postgrest bd integracion_cfisicas, ran
servicios_sig1:postgrest bd integracion_cfisicas, ran
servicios_sig1:postgrest bd integracion_cfisicas, ran
servicios_sig1:postgrest bd integracion_cfisicas, ran


In [12]:
settings_wms_viz

{'settings': {'zoom': 10, 'center': [-99.133209, 19.432608]},
 'skeleton': {'base': ['base_osm'],
  'main': [{'title': 'tamaulipas_prueba', 'layers': []},
   {'title': 'servicios_sig1', 'layers': []}]},
 'layers': [{'type': 'tile',
   'name': 'base_osm',
   'title': 'Open Street Map',
   'visible': True,
   'settings': {'ol_source_accesor': 'OSM'},
   'plugin_view': {}}]}

In [6]:
works_to_usar

[{'workspace': {'name': 'tamaulipas_prueba',
   'isolated': False,
   'dataStores': 'http://localhost:8080/geoserver/rest/workspaces/tamaulipas_prueba/datastores.json',
   'coverageStores': 'http://localhost:8080/geoserver/rest/workspaces/tamaulipas_prueba/coveragestores.json',
   'wmsStores': 'http://localhost:8080/geoserver/rest/workspaces/tamaulipas_prueba/wmsstores.json',
   'wmtsStores': 'http://localhost:8080/geoserver/rest/workspaces/tamaulipas_prueba/wmtsstores.json'},
  'list_dataStores': [{'dataStore': {'name': 'tamaulipas911_aut',
     'description': 'una base de datos de postgis',
     'type': 'PostGIS',
     'enabled': True,
     'workspace': {'name': 'tamaulipas_prueba',
      'href': 'http://localhost:8080/geoserver/rest/workspaces/tamaulipas_prueba.json'},
     'connectionParameters': {'entry': [{'@key': 'schema', '$': 'niv_muns'},
       {'@key': 'Evictor run periodicity', '$': '300'},
       {'@key': 'Max open prepared statements', '$': '50'},
       {'@key': 'encode 

In [ ]:
input()